In [ ]:
# ! pip install arxiv
# ! pip install wikipedia
! pip install langchainhub

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o",temperature=0)
llm

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=10,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki.name

In [ ]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
print(docs)
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith",
                                       "Search for langsmith. For any question about langsmith, you must use this tool.")

retriever_tool

In [ ]:
# Arxiv

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper)
arxiv.name

In [ ]:
"""
 Combine all the 3 tools:
 1. Wikipedia
 2. Langsmith - retrieval tool
 3. Arxiv
 """

tools = [wiki,retriever_tool,arxiv]
tools

Agents

In [ ]:
# from langchain import hub

# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


In [ ]:
# Define the agent and let the llm know about the tools it can access.

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)


In [ ]:
# Agent executor

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose =True)

In [ ]:
agent_executor.invoke({"input":"Who is self attention mechanism?"})